In [1]:
!pip install  InstructorEmbedding   duckduckgo-search langchain-core   huggingface_hub  sentence-transformers langchain-text-splitters  langchain-chroma    langchain_community  google-generativeai  langchain-google-genai    faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 17.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 115.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.8/160.8 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.0/760.0 kB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 78.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4

In [36]:
##### from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_chroma import Chroma
import chromadb.utils.embedding_functions as embedding_functions
from dotenv import load_dotenv
from langchain.docstore.document import Document
import os
import google.generativeai as genai
from duckduckgo_search import DDGS
from bs4 import BeautifulSoup
import requests
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from typing import Any, List, Optional

from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM
from langchain_core.outputs import GenerationChunk
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain.load import dumps, loads

from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
import torch
import faiss
import numpy as np
from datasets import load_dataset
from concurrent.futures import ThreadPoolExecutor



ds = load_dataset("HPAI-BSC/medqa-cot")

medqa_cot_embeds = np.load("medqa_cot.npy")

index = faiss.IndexFlatIP(768)
index.add(medqa_cot_embeds)

query_model = AutoModel.from_pretrained("ncbi/MedCPT-Query-Encoder")
query_tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Query-Encoder")

load_dotenv()


os.environ["GEMINI_API_KEY"] = ""


model_name = "hkunlp/instructor-xl"
model_kwargs = {"device": "cuda"}
encode_kwargs = {"normalize_embeddings": True}
instructor_embeddings = HuggingFaceInstructEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

tokenizer = AutoTokenizer.from_pretrained("ncbi/MedCPT-Cross-Encoder")
model = AutoModelForSequenceClassification.from_pretrained("ncbi/MedCPT-Cross-Encoder")


translate_model = genai.GenerativeModel("gemini-1.0-pro")  # buat translate


def translate_text(text, language):
    newText = text
    if language == "Indonesian":
        newText = text.replace("\n", " \n ")
    newText = (
        newText
        + f"; translate to {language}. please just translate the text and don't answer the questions!"
    )
    return translate_model.generate_content(newText).candidates[0].content.parts[0].text


summarizer_model = genai.GenerativeModel("gemini-1.5-flash-8b")


def user_summarizer(text):
    return summarizer_model.generate_content(text).candidates[0].content.parts[0].text


retriever_model = Chroma(
    collection_name="welllahh_rag_collection_chromadb",
    persist_directory="./chroma_langchain_db2",
    embedding_function=instructor_embeddings,
)

retriever = retriever_model.as_retriever(search_kwargs={"k": 10})


template = """Write multiple different very short search queries (each queries by a separated by "," & maximum different 4 very short search queries) that will help answer complex user questions, make sure in your answer you only give multiple different very short search queries (each queries by a separated by "," & maximum different 4 very short search queries) and don't include any other text! . Original question: {question}"""
search_query_prompt = ChatPromptTemplate.from_template(template)


class GeminiLLM(LLM):
    """custom model pakai gemini"""

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")
        llm = genai.GenerativeModel(
            model_name="tunedModels/gemini-welllahh-zerotemp-lrfv-3536"  # sebelumnya 0
        )  # buat jawab pertanyaan medis

        ans = (
            llm.generate_content(prompt, generation_config={"temperature": 0.12})
            .candidates[0]
            .content.parts[0]
            .text
        )
        return ans

    @property
    def _llm_type(self) -> str:
        """Get the type of language model used by this chat model. Used for logging purposes only."""
        return "gemini-welllahh-zerotemp-lrfv-3536"


llm = GeminiLLM()

generate_query = search_query_prompt | llm | StrOutputParser()

cant_access = {
    "npin.cdc.gov",
    "www.ncbi.nlm.nih.gov",
}  # gak bisa diakses & gak muncul tag <p> nya

def scrape_websearch(queryProc):
    websearch = []
    try:
        results = DDGS().text(queryProc, max_results=8)
    except Exception:
        return websearch
    for res in results:
        domain = res["href"].split("/")[2]
        if "webmd" in res["href"] or ".pdf" in res["href"] or domain in cant_access:
            continue
        if len(websearch) == 3:
            break
        if ".org" in res["href"] or ".gov" in res["href"] or "who" in res["href"]:

            link = res["href"]
            try:
                page = requests.get(link).text
            except requests.exceptions.RequestException as errh:
                print(f"error: {errh}")
                continue
            doc = BeautifulSoup(page, features="html.parser")
            text = ""
            hs = doc.find_all("h2")
            h3s = doc.find_all("h3")
            ps = doc.find_all("p")
            for h3 in h3s:
                hs.append(h3)
            for pp in ps:
                hs.append(pp)

            hs_parents = set()
            for h2 in hs:
                h2_parent = h2.parent
                if h2_parent in hs_parents:
                    continue
                hs_parents.add(h2_parent)
                h2_adjacent = h2_parent.children
                for adjacent in h2_adjacent:
                    if adjacent.name == "p" and adjacent.text != "\n":
                        text += adjacent.text + "\n"
                    if (
                        adjacent.name == "h2"
                        or adjacent.name == "h3"
                        or adjacent.name == "h4"
                    ):
                        text += adjacent.text + ": \n"
                    if adjacent.name == "ul" or adjacent.name == "ol":
                        text += ": "
                        for li in adjacent.find_all("li"):
                            text += li.text + ","
                        text += "\n"
            if "Why have I been blocked" in text or text == "" or text == ": \n":
                continue

            websearch.append(text)
    return websearch


def add_websearch_results(query):
    queries = []
    if "," in query:
        queries.extend(query.split(","))
    else:
        queries = [query]
    websearch_all = []

    with ThreadPoolExecutor(max_workers=2) as executor:
        results = executor.map(scrape_websearch, queries)
    for new_contexts in results:
        websearch_all.extend(new_contexts)
               
    return websearch_all


class DuckDuckGoRetriever(BaseRetriever):
    """ """

    """List of documents to retrieve from."""
    k: int
    """Number of top results to return"""

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Sync implementations for retriever."""
        matching_documents = []
        websearch = add_websearch_results(query)

        for document in websearch:
            if len(matching_documents) > self.k:
                return matching_documents

            matching_documents.append(document)
        return matching_documents


medqa_cot_data = ds["train"]




class MedQACoTRetriever(BaseRetriever):
    """List of documents to retrieve from."""

    k: int

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        """Sync implementations for retriever."""
        matching_documents = []
        # websearch = add_websearch_results(query)
        inds = []

        queries = [query]
        if "," in query:
            queries.extend(query.split(","))
        else:
            queries = [query]

        relevant_cot = []
        
        def retrieve_cot(search_query):
            with torch.no_grad():
                # tokenize the queries
                encoded = query_tokenizer(
                    [search_query],
                    truncation=True,
                    padding=True,
                    return_tensors="pt",
                    max_length=512,
                )
                embeds = query_model(**encoded).last_hidden_state[:, 0, :]
                scores, inds = index.search(embeds, k=self.k)

            curr_relevant_cot = []
            for score, ind in zip(scores[0], inds[0]):
                curr_cot = medqa_cot_data[int(ind)]
                curr_question = curr_cot["question"]
                curr_answer = curr_cot["response"]
                curr_relevant_cot.append(
                    f"\nQuestion: {curr_question}\nAnswer: Let's think step by step. {curr_answer}"
                )
            return curr_relevant_cot
        
        
        with ThreadPoolExecutor(max_workers=3) as executor:
            results = executor.map(retrieve_cot, queries)
        for new_contexts in results:
            relevant_cot.extend(new_contexts)

        return relevant_cot


websearch_retriever = DuckDuckGoRetriever(k=2)
medqa_cot_retriever = MedQACoTRetriever(k=1)



def rerank_docs_medcpt(queries, docs):
    queries = [queries]
    queries.extend(queries[0].split(","))
    relevant = set()

    def process_query(query):
        pairs = [[query, article] for article in docs]
        with torch.no_grad():
            encoded = tokenizer(
                pairs,
                truncation=True,
                padding=True,
                return_tensors="pt",
                max_length=512,
            )

            logits = model(**encoded).logits.squeeze(dim=1)
            values, indices = torch.sort(logits, descending=True)
            curr_relevant = [docs[i] for i in indices[:3]]
        return curr_relevant
    
    with ThreadPoolExecutor(max_workers=3) as executor:
        results = executor.map(process_query, queries)

    for curr_relevant in results:
        relevant.update(curr_relevant)
                
    relevant = list(relevant)
    return relevant


retrieval_chain = generate_query | {
    "websearch": websearch_retriever,
    "medqa_cot": medqa_cot_retriever,
    "query": StrOutputParser(),
}


def format_docs(docs):
    query = docs["query"]
    chroma_docs = [doc.metadata["content"] + doc.page_content for doc in docs["chroma"]]
    relevant_cot = docs["medqa_cot"]

    docs = list(set(chroma_docs + docs["websearch"] ))
    # rerank passage2 dari document chromadb & hasil scraping webpage hasil duckduckgosearch
    relevant_docs = rerank_docs_medcpt(query, docs)
    context = " \n\n".join(doc for doc in relevant_docs)

    # tambahin few-shot chain-of-thought prompting
    relevant_cot = list(set(relevant_cot))[:2]
    for cot in relevant_cot:
        context += " \n" + cot
    return context, relevant_docs


template = """Answer the question based only on the following context:
{context}
\n

please do not mention the same answer more than once and Don't say 'the given text does not answer the user's question or is not relevant to the user's question' in your answer. just answer the question don't add any other irrelevant text
\n
Question: {question}
\n
Answer: Let's think step by step.
"""

prompt = ChatPromptTemplate.from_template(template)

answer_chain = (
     prompt
    | llm
    | {"llm_output": StrOutputParser()}
)


def retrieve_and_append(query):
    new_knowledge_base_contexts = retriever.invoke(query)
    return new_knowledge_base_contexts
    
def answer(question):
    docs = retrieval_chain.invoke(question)
    docs["chroma"] = []
    search_query = [docs["query"]]
    search_query.extend(docs["query"].split(","))
   
    with ThreadPoolExecutor(max_workers=1) as executor:
        results = executor.map(retrieve_and_append, search_query)
    for new_contexts in results:
        docs["chroma"].extend(new_contexts)
    
    context, relevant_docs = format_docs(docs)
    answer = answer_chain.invoke({"context": context, "question": question})

    context = f"search query: {','.join(search_query)}\n" + context
    return answer, context, [context] + relevant_docs




def answer_pipeline_for_eval(question):
    """
    buat evaluasi llm.
    Pertanyaan dalam bahasa inggris.
    pakai pertanyaan dari PubmedQA, MedQA -> pakai  (exact match)
    pakai: https://huggingface.co/datasets/GBaker/MedQA-USMLE-4-options/viewer/default/test?row=23
    
    """
    question = question.replace("\n", "  ")
    # print("retrieving relevant passages and answering user question....")
    pred, context, relevant_docs  = answer(question)
    return pred["llm_output"], context, relevant_docs



load INSTRUCTOR_Transformer
max_seq_length  512


In [2]:
genai.configure(api_key="PUNYAMU")

In [3]:
eval_ds_medqa = load_dataset("GBaker/MedQA-USMLE-4-options")


In [4]:
eval_ds_medqa = eval_ds_medqa["test"]

In [5]:
eval_ds_medqa[0]["answer"]

'Tell the attending that he cannot fail to disclose this mistake'

In [6]:
llm_answer_data = [{ "question": '',
                "llm_answer": '',
                "llm_answer_choice":'',
                "retrieved_contexts": '',
                "reference_choice": '',
                "reference_answer": '',}] * len(eval_ds_medqa)


In [7]:
medqa_question = []

In [8]:
for test in eval_ds_medqa:
    question_with_choice = test["question"] + " Choices: "
    question_with_choice += "A. " + test["options"]["A"]+ ", "
    question_with_choice += "B. " + test["options"]["B"]+ ", "
    question_with_choice += "C. " + test["options"]["C"]+ ", "
    question_with_choice += "D. " + test["options"]["D"]+ ", "
    medqa_question.append(question_with_choice)
    
    
    

In [9]:
max_retries=10

In [10]:
import re
import pandas as pd

def locate_answer(answer:str):
    pattern = r"([Aa]nswer):(.*)"

    matches = re.findall(pattern, answer)
    choice = matches[0][1]
    

    ans = re.findall("^\s*(A|B|C|D)$", choice)
    if len(ans) > 0:
        return ans[0].upper()
    
    ans = re.findall("^\s*(A|B|C|D) or", choice)
    if len(ans) > 0:
        return ans[0].upper()
    
    ans = re.findall("^\s*(A|B|C|D) and", choice)
    if len(ans) > 0:
        return ans[0].upper()
        
    ans = re.findall("^\s*(A|B|C|D)/", choice)
    if len(ans) > 0:
        return ans[0].upper()
    
    ans = re.findall("^\s*(A|B|C|D),", choice)
    if len(ans) > 0:
        return ans[0].upper()
    
    ans = re.findall("[Oo]ption (A|B|C|D)", choice)
    if len(ans) > 0:
        return ans[0]

    ans = re.findall(":\s*(A|B|C|D)", choice)
    if len(ans) > 0:
        return ans[0].upper()

    ans = re.findall("^\s*(A|B|C|D)\.", choice)
    if len(ans) > 0:
        return ans[0].upper()

    ans = re.findall("^\s*(A|B|C|D)\"", choice)
    if len(ans) > 0:
        return ans[0].upper()
    
    ans = re.findall("^\s*(A|B|C|D):", choice)
    if len(ans) > 0:
        return ans[0].upper()

    return "Not Answered"

In [11]:
csv_file = "llm_answer_data_th2.csv"


In [12]:
curr_question_idx_tracker = 0


In [ ]:
import traceback

import time
for curr_question_idx in range(curr_question_idx_tracker, len(medqa_question)):
    question = medqa_question[curr_question_idx]
    retry_count = 0 
    while retry_count < max_retries:
        try:
            llm_answer, context, relevant_docs = answer_pipeline_for_eval(question)
            
            gold_answer_choice = eval_ds_medqa[curr_question_idx]["answer_idx"]
            gold_answer = eval_ds_medqa[curr_question_idx]["answer"]
            print(f"selesai answer question ke-{curr_question_idx} \nGemini Answer: {locate_answer(llm_answer)}\nGold Answer: {gold_answer_choice}. {gold_answer}")
            llm_answer_data[curr_question_idx] = {
                "question": question,
                "llm_answer": llm_answer,
                "llm_answer_choice": locate_answer(llm_answer),
                "retrieved_contexts": relevant_docs,
                "reference_choice": gold_answer_choice,
                "reference_answer": gold_answer,
                
            }
            
            pd.DataFrame(llm_answer_data).to_csv(csv_file)
            
            break
        except Exception as e:
            retry_count += 1
            
            time.sleep(7)
            if retry_count == max_retries:
                print(f"failed/error on question: {curr_question_idx}")
                break
    curr_question_idx_tracker += 1

retrieving relevant passages and answering user question....
selesai answer question ke-0 
Gemini Answer: A
Gold Answer: B. Tell the attending that he cannot fail to disclose this mistake
retrieving relevant passages and answering user question....
selesai answer question ke-1 
Gemini Answer: B
Gold Answer: D. Cross-linking of DNA
retrieving relevant passages and answering user question....
selesai answer question ke-2 
Gemini Answer: B
Gold Answer: B. Cholesterol embolization
retrieving relevant passages and answering user question....
error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
error: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
selesai answer question ke-3 
Gemini Answer: A
Gold Answer: D. Lactose-fermenting, gram-negative rods forming pink colonies on MacConkey agar
retrieving relevant passages and answering user question....
selesai answer question ke-4 
Gemini Answer: B
Gold An

In [17]:
def evaluate(curr_question_idx_tracker):
    for curr_question_idx in range(curr_question_idx_tracker, len(medqa_question)):
        question = medqa_question[curr_question_idx]
        retry_count = 0 
        while retry_count < max_retries:
            try:
                llm_answer, context, relevant_docs = answer_pipeline_for_eval(question)

                gold_answer_choice = eval_ds_medqa[curr_question_idx]["answer_idx"]
                gold_answer = eval_ds_medqa[curr_question_idx]["answer"]
                print(f"selesai answer question ke-{curr_question_idx} \nGemini Answer: {locate_answer(llm_answer)}\nGold Answer: {gold_answer_choice}. {gold_answer}")
                llm_answer_data[curr_question_idx] = {
                    "question": question,
                    "llm_answer": llm_answer,
                    "llm_answer_choice": locate_answer(llm_answer),
                    "retrieved_contexts": relevant_docs,
                    "reference_choice": gold_answer_choice,
                    "reference_answer": gold_answer,

                }

                pd.DataFrame(llm_answer_data).to_csv(csv_file)

                break
            except Exception as e:
                # print(f"error: {e}")
                if retry_count % 15 == 0:
                    print(f"error : {e}")
                retry_count += 1

                time.sleep(7)
                if retry_count == max_retries:
                    print(f"failed/error on question: {curr_question_idx}")
                    break
        curr_question_idx_tracker += 1

In [ ]:
evaluate(127)

retrieving relevant passages and answering user question....
selesai answer question ke-127 
Gemini Answer: C
Gold Answer: C. Natural killer cells
retrieving relevant passages and answering user question....
selesai answer question ke-128 
Gemini Answer: D
Gold Answer: A. Intrafascicular infiltration on muscle biopsy
retrieving relevant passages and answering user question....
selesai answer question ke-129 
Gemini Answer: A
Gold Answer: A. Decreased acetylcholine release
retrieving relevant passages and answering user question....
selesai answer question ke-130 
Gemini Answer: B
Gold Answer: B. Biliary atresia
retrieving relevant passages and answering user question....
selesai answer question ke-131 
Gemini Answer: B
Gold Answer: B. QT prolongation
retrieving relevant passages and answering user question....
selesai answer question ke-132 
Gemini Answer: A
Gold Answer: A. Diverticulitis
retrieving relevant passages and answering user question....
retrieving relevant passages and answ

In [16]:
curr_question_idx_tracker = 234

In [28]:
llm_answer_data = pd.read_csv("llm_answer_data_th2.csv")

In [34]:
# llm_answer_data = llm_answer_data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [29]:
llm_answer_data = llm_answer_data.to_dict('records')

In [ ]:
import time

evaluate(curr_question_idx_tracker)

retrieving relevant passages and answering user question....
selesai answer question ke-234 
Gemini Answer: D
Gold Answer: C. Secretion of γ-interferon by activated T-lymphocytes
retrieving relevant passages and answering user question....
selesai answer question ke-235 
Gemini Answer: A
Gold Answer: A. Active hepatitis A infection
retrieving relevant passages and answering user question....
selesai answer question ke-236 
Gemini Answer: A
Gold Answer: A. Nucleotide excision repair
retrieving relevant passages and answering user question....
selesai answer question ke-237 
Gemini Answer: D
Gold Answer: B. Dense fibroinflammatory infiltrate
retrieving relevant passages and answering user question....
retrieving relevant passages and answering user question....
selesai answer question ke-238 
Gemini Answer: A
Gold Answer: A. Incidence rates will be higher during shorter time periods than longer periods.
retrieving relevant passages and answering user question....
selesai answer question 

In [39]:
# coba eval current progress
th2 = pd.read_csv("llm_answer_data_th2.csv")
correct = 0
num_data = 0
for idx, row in th2.iterrows():
    if row["llm_answer_choice"] == row["reference_choice"]:
        correct +=1
    if row["reference_choice"] in {"A", "B", "C", "D"}:
        num_data +=1

print(f"medqa 300 row accuracy: {(correct/num_data)*100}")
    

medqa 300 row accuracy: 70.54545454545455


In [40]:
curr_question_idx_tracker = 304

In [ ]:
import time

evaluate(curr_question_idx_tracker)

retrieving relevant passages and answering user question....
selesai answer question ke-304 
Gemini Answer: A
Gold Answer: A. Asthma attack
retrieving relevant passages and answering user question....
selesai answer question ke-305 
Gemini Answer: B
Gold Answer: B. Lactulose
retrieving relevant passages and answering user question....
selesai answer question ke-306 
Gemini Answer: D
Gold Answer: C. Ehlers-Danlos syndrome
retrieving relevant passages and answering user question....
error: HTTPSConnectionPool(host='www.guides.loc.gov', port=443): Max retries exceeded with url: /case-law/citations (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fb312494190>: Failed to resolve 'www.guides.loc.gov' ([Errno -5] No address associated with hostname)"))
selesai answer question ke-307 
Gemini Answer: B
Gold Answer: C. Contact another family member for consent
retrieving relevant passages and answering user question....
selesai answer question ke-308 
Gemini Answe

In [ ]:
curr_question_idx_tracker = 413

import time

evaluate(curr_question_idx_tracker)

selesai answer question ke-413 
Gemini Answer: B
Gold Answer: A. 0.9% saline IV
selesai answer question ke-414 
Gemini Answer: D
Gold Answer: D. Enterotoxigenic Escherichia coli
selesai answer question ke-415 
Gemini Answer: D
Gold Answer: D. If the sensitivity of this screening test were decreased, the statistical power would decrease.
error : list index out of range
error : list index out of range
error : list index out of range
failed/error on question: 416
selesai answer question ke-417 
Gemini Answer: A
Gold Answer: A. Inadvertent surgical removal of parathyroid glands
selesai answer question ke-418 
Gemini Answer: C
Gold Answer: C. Iron supplementation
selesai answer question ke-419 
Gemini Answer: D
Gold Answer: B. Protein C concentrate
selesai answer question ke-420 
Gemini Answer: B
Gold Answer: B. Long face and large everted ears
selesai answer question ke-421 
Gemini Answer: C
Gold Answer: C. Metoprolol
selesai answer question ke-422 
Gemini Answer: D
Gold Answer: A. Increas

In [4]:
# coba eval current progress
import pandas as pd
th2 = pd.read_csv("llm_answer_data_th2.csv")
correct = 0
num_data = 0
for idx, row in th2.iterrows():
    if row["llm_answer_choice"] == row["reference_choice"]:
        correct +=1
    if row["reference_choice"] in {"A", "B", "C", "D"}:
        num_data +=1

print(f"medqa 400 row accuracy: {(correct/num_data)*100}")
#https://paperswithcode.com/sota/question-answering-on-medqa-usmle

medqa 400 row accuracy: 70.12658227848101


In [16]:
from sklearn.metrics import accuracy_score
y_true = ["N/A"]* num_data
y_pred = ["N/A"] * num_data

abcd_to_idx = {"A": 0, "B": 1, "C": 2, "D": 3, "N/A": 4, 'Not Answered': 5}

nn_idx = 0
for idx, row in th2.iterrows():       
    if row["reference_choice"] in {"A", "B", "C", "D"}:
        y_true[nn_idx] = abcd_to_idx[row["llm_answer_choice"]]
        y_pred[nn_idx] = abcd_to_idx[row["reference_choice"] ]
        nn_idx += 1
    

In [20]:
from sklearn.metrics import f1_score

print(accuracy_score(y_true, y_pred)*100)
f1_score(y_true, y_pred, average='macro')

70.12658227848101


0.5605519269187214